#### Training GraphSAINT

Model using the graph dataset extracted from the csv file. TO generate csv file, please run the parsing notebook. 

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
import dgl.nn as dglnn
from dgl.dataloading import SAINTSampler
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# ------------------------------
# 1. Load Dataset and Prepare Data
# ------------------------------
df = pd.read_csv("all_circuits_features.csv")
print("\n?? Dataset Overview:")
print(df.info())

label_encoder = LabelEncoder()
df["gate_label"] = label_encoder.fit_transform(df["gate_type"])

feature_columns = [
    "fan_in", "fan_out", "dist_to_output", "is_primary_input", "is_primary_output",
    "is_internal", "is_key_gate", "degree_centrality", "betweenness_centrality",
    "closeness_centrality", "clustering_coefficient", "avg_fan_in_neighbors", "avg_fan_out_neighbors"
]

df = df.dropna(subset=feature_columns)
df[feature_columns] = df[feature_columns].astype(float)

scaler = StandardScaler()
df[feature_columns] = scaler.fit_transform(df[feature_columns])

# ---------------------------
# 2. Build Node and Edge Lists
# ---------------------------
nodes = df["node"].tolist()
node_to_id = {node: i for i, node in enumerate(nodes)}

edges = []
for _, row in df.iterrows():
    node_id = node_to_id[row["node"]]
    potential_sources = df[df["fan_out"] > 0]["node"].tolist()
    num_fan_in = int(row["fan_in"])
    if num_fan_in > 0:
        sources = potential_sources[:num_fan_in]
        for src in sources:
            if src in node_to_id:
                edges.append((node_to_id[src], node_id))

print("\n?? Extracted", len(edges), "edges.")
if len(edges) == 0:
    raise ValueError("No edges found! Check your fan_in values.")

src_nodes, dst_nodes = zip(*edges) if edges else ([], [])
src_tensor = torch.tensor(src_nodes, dtype=torch.int64)
dst_tensor = torch.tensor(dst_nodes, dtype=torch.int64)
valid_edges = (
    (src_tensor >= 0) & (dst_tensor >= 0) &
    (src_tensor < len(nodes)) & (dst_tensor < len(nodes))
)
src_tensor = src_tensor[valid_edges]
dst_tensor = dst_tensor[valid_edges]

# ---------------------------
# 3. Create the DGL Graph
# ---------------------------
graph = dgl.graph((src_tensor, dst_tensor), num_nodes=len(nodes))
graph = dgl.add_self_loop(graph)

graph.ndata['features'] = torch.tensor(df[feature_columns].values, dtype=torch.float32)
graph.ndata['labels'] = torch.tensor(df["gate_label"].values, dtype=torch.long)

nodes_idx = np.arange(len(nodes))
train_idx, test_idx = train_test_split(nodes_idx, test_size=0.2, random_state=42)
train_nid = torch.tensor(train_idx, dtype=torch.long)
test_nid = torch.tensor(test_idx, dtype=torch.long)

# ---------------------------
# 4. Define the GraphSAINT Model
# ---------------------------
class GraphSAINTModel(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super().__init__()
        self.conv1 = dglnn.GraphConv(in_feats, hidden_feats, allow_zero_in_degree=True)
        self.conv2 = dglnn.GraphConv(hidden_feats, out_feats, allow_zero_in_degree=True)

    def forward(self, g, x):
        x = self.conv1(g, x)
        x = torch.relu(x)
        x = self.conv2(g, x)
        return x

in_feats = len(feature_columns)
hidden_feats = 32
out_feats = len(label_encoder.classes_)
model = GraphSAINTModel(in_feats, hidden_feats, out_feats)

# ---------------------------
# 5. GraphSAINT Sampler & Training Loop
# ---------------------------
saint_mode = 'walk'         # 'walk', 'node', or 'edge'
saint_budget = (3000, 2)    # For 'walk': (num_roots, walk_length)
num_traversals = 30         # subgraphs per epoch

sampler = SAINTSampler(
    mode=saint_mode,
    budget=saint_budget
)

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    steps = 0

    for _ in range(num_traversals):
        subg = sampler.sample(graph, train_nid)

        # Materialize features/labels from parent graph
        nids = subg.ndata[dgl.NID]
        feats = graph.ndata['features'][nids]
        labels = graph.ndata['labels'][nids]

        logits = model(subg, feats)
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        steps += 1

    avg_loss = total_loss / max(steps, 1)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, SAINT avg loss: {avg_loss:.4f}")

# ---------------------------
# 6. Evaluation on Full Graph
# ---------------------------
model.eval()
with torch.no_grad():
    logits = model(graph, graph.ndata['features'])
    test_logits = logits[test_nid]
    test_predictions = test_logits.argmax(dim=1)
    orig_accuracy = (test_predictions == graph.ndata['labels'][test_nid]).float().mean().item()

print(f"\n? Test Accuracy on Original Graph: {orig_accuracy * 100:.2f}%")

true_labels_orig = graph.ndata['labels'][test_nid].cpu().numpy()
pred_labels_orig = test_predictions.cpu().numpy()
conf_mat_orig = confusion_matrix(true_labels_orig, pred_labels_orig)

print("\n?? Confusion Matrix (Original Graph):")
print(conf_mat_orig)

print("\nClassification Report (Original Graph):")
print(classification_report(true_labels_orig, pred_labels_orig, target_names=label_encoder.classes_))



?? Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60882 entries, 0 to 60881
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   circuit_name            60882 non-null  object 
 1   node                    60882 non-null  object 
 2   gate_type               60882 non-null  object 
 3   fan_in                  60882 non-null  int64  
 4   fan_out                 60882 non-null  int64  
 5   depth                   60882 non-null  object 
 6   dist_to_output          60882 non-null  int64  
 7   is_primary_input        60882 non-null  int64  
 8   is_primary_output       60882 non-null  int64  
 9   is_internal             60882 non-null  int64  
 10  is_key_gate             60882 non-null  int64  
 11  key_dependency          122 non-null    object 
 12  degree_centrality       60882 non-null  float64
 13  betweenness_centrality  60882 non-null  float64
 14  closeness_centra

#### Jacobian 

In [9]:
import torch
import numpy as np

print("\n--- Jacobian Computation and Relative Error (GraphSAINT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
device = graph.ndata['features'].device
labels_np = graph.ndata['labels'].cpu().numpy()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid (index array)
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts (selected)
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Helper: f(x) returns logits for node test_idx with its feature vector replaced by x
# --------------------------------
def node_logits_with_replaced_features(x, test_idx):
    # x: [F] tensor on device
    new_features = graph.ndata['features'].clone()
    new_features[test_idx] = x
    out = model(graph, new_features)
    return out[test_idx]  # shape: [num_classes]

# --------------------------------
# 3) Main loop: Jacobian + finite-difference verification
# --------------------------------
class_metrics = {cn: {'jacobian_norms': [], 'relative_errors': []} for cn in class_names}
overall_jacobian_norms = []
overall_relative_errors = []

epsilon = 1e-3  # small perturbation for FD check

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Base feature vector for this node
    x0 = graph.ndata['features'][test_idx].clone().detach().to(device).requires_grad_(True)

    # Define f(x) for autograd Jacobian (must not use no_grad)
    def f(x):
        return node_logits_with_replaced_features(x, test_idx)

    # Compute Jacobian J (num_classes x feat_dim)
    J = torch.autograd.functional.jacobian(f, x0)  # shape: [C, F]

    # Frobenius norm of the Jacobian
    jacobian_norm = torch.norm(J, p='fro').item()

    # Finite-difference verification
    delta = epsilon * torch.randn_like(x0)
    predicted_change = J.mv(delta)  # [C]
    f_x0 = f(x0)
    f_x0_perturbed = f(x0 + delta)
    actual_change = f_x0_perturbed - f_x0
    rel_error = (torch.norm(predicted_change - actual_change) /
                 (torch.norm(actual_change) + 1e-8)).item()

    # Store metrics
    class_metrics[class_name]['jacobian_norms'].append(jacobian_norm)
    class_metrics[class_name]['relative_errors'].append(rel_error)
    overall_jacobian_norms.append(jacobian_norm)
    overall_relative_errors.append(rel_error)

# --------------------------------
# 4) Reporting
# --------------------------------
print("\nClass-wise Jacobian Analysis Results:")
header = "{:<12s} {:>30s} {:>35s}".format("Class", "Avg. Jacobian Norm ± Std", "Avg. Relative Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    norms = class_metrics[cn]['jacobian_norms']
    errors = class_metrics[cn]['relative_errors']
    avg_norm = np.mean(norms) if norms else 0.0
    std_norm = np.std(norms) if norms else 0.0
    avg_rel_error = np.mean(errors) if errors else 0.0
    std_rel_error = np.std(errors) if errors else 0.0
    print("{:<12s} {:>15.4f} ± {:<12.4f} {:>15.4e} ± {:<10.4e}".format(
        cn, avg_norm, std_norm, avg_rel_error, std_rel_error
    ))

# Overall aggregates
overall_avg_norm = float(np.mean(overall_jacobian_norms)) if overall_jacobian_norms else 0.0
overall_std_norm = float(np.std(overall_jacobian_norms)) if overall_jacobian_norms else 0.0
overall_avg_rel_error = float(np.mean(overall_relative_errors)) if overall_relative_errors else 0.0
overall_std_rel_error = float(np.std(overall_relative_errors)) if overall_relative_errors else 0.0

print("\nOverall Aggregated Jacobian Analysis Results:")
print("Average Jacobian Norm: {:.4f} ± {:.4f}".format(overall_avg_norm, overall_std_norm))
print("Average Relative Error: {:.4e} ± {:.4e}".format(overall_avg_rel_error, overall_std_rel_error))



--- Jacobian Computation and Relative Error (GraphSAINT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Jacobian Analysis Results:
Class              Avg. Jacobian Norm ± Std           Avg. Relative Error ± Std
-------------------------------------------------------------------------------
and                  45.7088 ± 13.3837           1.1304e-04 ± 2.9016e-04
input                27.1540 ± 3.0831            1.8050e-02 ± 1.0712e-01
nand                 22.2573 ± 7.0761            2.6081e-03 ± 1.6647e-02
nor                  22.7983 ± 3.6259        

#### Local Lipschitz constant

In [10]:
import torch
import numpy as np

print("\n--- Local Lipschitz Constant & Relative Error (GraphSAINT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid (index array)
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts (selected)
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Helper: f(x) returns logits for node test_idx with its feature vector replaced by x
# --------------------------------
def node_logits_with_replaced_features(x, test_idx):
    new_features = graph.ndata['features'].clone()
    new_features[test_idx] = x
    out = model(graph, new_features)
    return out[test_idx]  # [num_classes]

# --------------------------------
# 3) Main loop: Local Lipschitz (||J||_2) + finite-difference relative error
# --------------------------------
class_lipschitz = {cn: [] for cn in class_names}
class_rel_errors = {cn: [] for cn in class_names}
overall_lipschitz = []
overall_rel_errors = []

epsilon = 1e-3
trials_per_node = 10

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Base feature vector
    x0 = graph.ndata['features'][test_idx].clone().detach().requires_grad_(True)

    # Define scalar-to-vector function for Jacobian
    def f(x):
        return node_logits_with_replaced_features(x, test_idx)

    # Jacobian J: [C, F]
    J = torch.autograd.functional.jacobian(f, x0)

    # Local Lipschitz constant: operator 2-norm (largest singular value)
    L_local = torch.linalg.norm(J, ord=2).item()

    # Finite-difference relative error across multiple random directions
    rel_errors_for_node = []
    f_x0 = f(x0).detach()
    for _ in range(trials_per_node):
        delta = epsilon * torch.randn_like(x0)
        # Predicted change norm via Lipschitz bound
        pred_change_norm = L_local * torch.norm(delta).item()
        # Actual change norm
        f_x0_pert = f(x0 + delta).detach()
        actual_change_norm = torch.norm(f_x0_pert - f_x0).item()
        rel_err = abs(pred_change_norm - actual_change_norm) / (actual_change_norm + 1e-8)
        rel_errors_for_node.append(rel_err)

    avg_rel_error_node = float(np.mean(rel_errors_for_node))

    # Store
    class_lipschitz[class_name].append(L_local)
    class_rel_errors[class_name].append(avg_rel_error_node)
    overall_lipschitz.append(L_local)
    overall_rel_errors.append(avg_rel_error_node)

# --------------------------------
# 4) Reporting
# --------------------------------
print("\nClass-wise Local Lipschitz (with Relative Errors):")
header = "{:<12s} {:>25s} {:>30s}".format("Class", "Avg Lipschitz ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    L_vals = class_lipschitz[cn]
    E_vals = class_rel_errors[cn]
    if L_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(L_vals), np.std(L_vals), np.mean(E_vals), np.std(E_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_lipschitz:
    print("\nOverall Aggregated:")
    print("Avg Lipschitz Constant: {:.4f} ± {:.4f}".format(np.mean(overall_lipschitz), np.std(overall_lipschitz)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated:")
    print("No samples selected; overall metrics unavailable.")



--- Local Lipschitz Constant & Relative Error (GraphSAINT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Local Lipschitz (with Relative Errors):
Class              Avg Lipschitz ± Std           Avg Rel. Error ± Std
---------------------------------------------------------------------
and              39.9835 ± 13.4321         3.2278e+00 ± 1.0483e+00
input            21.0064 ± 2.9048          2.5257e+00 ± 6.1535e-01
nand             15.8666 ± 6.4651          2.2312e+00 ± 6.6427e-01
nor              15.5405 ± 2.1582          1.9631e+00 ± 4.9402e-01
n

#### Hessian-Based Curvature Measure

In [16]:
import torch
import numpy as np
import dgl

print("\n--- Fast Hessian-Based Curvature (Grad Outer-Product, double precision) ---\n")

# -------------------------------
# 0) Config
# -------------------------------
max_per_class = 100
rng = np.random.default_rng(42)

epsilon_fd = 5e-3         # step for finite-difference check
trials_per_node = 3       # reduce trials for speed
power_iters = 8           # kept for compatibility (unused here)
khop_enable = True
khop_K = 2                # for 2-layer GNNs, K=2 covers dependency
dtype_eval = torch.float64

# -------------------------------
# 1) Setup
# -------------------------------
model.eval()

class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

device = graph.ndata['features'].device
orig_dtype = next(model.parameters()).dtype if any(p.requires_grad for p in model.parameters()) else torch.float32

# Cast model and features to double for stable curvature; restore later
model = model.to(dtype=dtype_eval, device=device)
orig_feats = graph.ndata['features']
graph.ndata['features'] = graph.ndata['features'].to(dtype_eval)

# Precompute logits once (full graph) for predicted class
with torch.no_grad():
    base_logits_full = model(graph, graph.ndata['features']).detach()

# -------------------------------
# 2) Select up to max_per_class test nodes per class
# -------------------------------
test_indices = test_nid.cpu().numpy()
selected = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    if len(idxs) == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if len(idxs) > max_per_class:
        chosen = rng.choice(idxs, size=max_per_class, replace=False).tolist()
        print(f"- {class_names[c]}: picked {max_per_class} from {len(idxs)} available.")
    else:
        chosen = idxs
        print(f"- {class_names[c]}: only {len(idxs)} available, taking all.")
    selected.extend(chosen)

selected = np.array(selected, dtype=np.int64)

print("\nFinal counts in selected set:")
for c in range(num_classes):
    cnt = int(np.sum(labels_np[selected] == c))
    print(f"- {class_names[c]}: {cnt}")

# -------------------------------
# 3) Caching k-hop subgraphs
# -------------------------------
subgraph_cache = {}

def get_local_subgraph(g, node_idx, K):
    if not khop_enable:
        return g, torch.arange(g.num_nodes(), device=device), int(node_idx)
    key = (int(node_idx), K)
    if key in subgraph_cache:
        return subgraph_cache[key]
    
    subg, nids = dgl.khop_in_subgraph(g, int(node_idx), k=K)
    # Ensure target node is present; if not, fall back to full graph
    match_idx = (nids == int(node_idx)).nonzero(as_tuple=False)
    if match_idx.numel() == 0:
        # Fallback: use full graph to avoid crash
        subgraph_cache[key] = (g, torch.arange(g.num_nodes(), device=device), int(node_idx))
        return subgraph_cache[key]
    
    local_target = match_idx.view(-1).item()
    subg = subg.to(device)
    nids = nids.to(device)
    subgraph_cache[key] = (subg, nids, local_target)
    return subgraph_cache[key]


# -------------------------------
# 4) h(x) and grad-outer-product curvature
# -------------------------------
def h_scalar(subg, x_local, local_to_global_nids, local_target, pred_class):
    """
    h(x) = log prob of 'pred_class' for the target node.
    Build feats as a differentiable function of x_local (no in-place assign).
    """
    feats0 = graph.ndata['features'][local_to_global_nids].to(dtype=x_local.dtype, device=x_local.device)
    delta = x_local - feats0[local_target]
    selector = torch.zeros(feats0.size(0), dtype=x_local.dtype, device=x_local.device)
    selector[local_target] = 1.0
    feats = feats0 + selector.unsqueeze(1) * delta.unsqueeze(0)

    logits = model(subg, feats)[local_target]
    logits = logits.squeeze(0)  # handles (C,) or (1,C)
    return torch.log_softmax(logits, dim=-1)[pred_class]

def grad_outer_prod_lambda_max(subg, x0, local_to_global_nids, local_target, pred_class):
    """
    Approximate largest Hessian eigenvalue by squared L2 norm of gradient:
    lambda_max  ||?_x h(x0)||²
    """
    x0_req = x0.detach().clone().requires_grad_(True)
    y = h_scalar(subg, x0_req, local_to_global_nids, local_target, pred_class)
    grad = torch.autograd.grad(y, x0_req, retain_graph=False, create_graph=False)[0]
    return grad.norm(p=2).item() ** 2, grad.detach(), y.detach()

# -------------------------------
# 5) Main loop: curvature + relative error (using GaussNewton approx)
# -------------------------------
class_curv = {cn: [] for cn in class_names}
class_relerr = {cn: [] for cn in class_names}
overall_curv, overall_rel = [], []

for test_idx in selected:
    label_idx = int(labels_np[int(test_idx)])
    class_name = class_names[label_idx]

    # Predicted class from precomputed logits
    pred_class = int(torch.argmax(base_logits_full[int(test_idx)]).item())

    # Local subgraph
    subg, local_to_global, local_target = get_local_subgraph(graph, int(test_idx), khop_K)

    # Local feature vector
    x0 = graph.ndata['features'][local_to_global[local_target]].detach().clone().to(dtype_eval).to(device)

    # Curvature via gradient outer-product: lambda_max  ||grad||^2
    lambda_max, g, y0_val = grad_outer_prod_lambda_max(subg, x0, local_to_global, local_target, pred_class)

    # Relative error of quadratic approximation using H  g g^T:
    # predicted second-order term: 0.5 * (g^T delta)^2
    errs = []
    for _ in range(trials_per_node):
        delta = epsilon_fd * torch.randn_like(x0)
        # Predicted second-order term under rank-1 Hessian approx
        gt_delta = torch.dot(g, delta).item()
        pred_second = 0.5 * (gt_delta ** 2)

        # Actual second-order remainder from Taylor expansion
        actual_second = (h_scalar(subg, x0 + delta, local_to_global, local_target, pred_class) - y0_val - torch.dot(g, delta)).item()
        rel = abs(pred_second - actual_second) / (abs(actual_second) + 1e-12)
        errs.append(rel)

    class_curv[class_name].append(lambda_max)
    class_relerr[class_name].append(float(np.mean(errs)))
    overall_curv.append(lambda_max)
    overall_rel.append(float(np.mean(errs)))

# -------------------------------
# 6) Reporting
# -------------------------------
print("\nClass-wise Hessian-Based Curvature (Grad Outer-Product) with Relative Errors:")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Max Eigenvalue ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))
for cn in class_names:
    vals = class_curv[cn]
    errs = class_relerr[cn]
    if vals:
        print("{:<12s} {:>11.6f} ± {:<10.6f} {:>15.4e} ± {:<10.4e}".format(
            cn, float(np.mean(vals)), float(np.std(vals)),
            float(np.mean(errs)), float(np.std(errs))
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_curv:
    print("\nOverall Aggregated Hessian-Based Curvature (Grad Outer-Product):")
    print("Avg Max Eigenvalue: {:.6f} ± {:.6f}".format(float(np.mean(overall_curv)), float(np.std(overall_curv))))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(float(np.mean(overall_rel)), float(np.std(overall_rel))))
else:
    print("\nOverall Aggregated Hessian-Based Curvature (Grad Outer-Product):")
    print("No samples selected; overall metrics unavailable.")

# -------------------------------
# 7) Cleanup: restore original types
# -------------------------------
graph.ndata['features'] = orig_feats
model = model.to(dtype=orig_dtype, device=device)



--- Fast Hessian-Based Curvature (Grad Outer-Product, double precision) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Hessian-Based Curvature (Grad Outer-Product) with Relative Errors:
Class                Avg. Max Eigenvalue ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and             2.209153 ± 8.051845        1.0515e+00 ± 1.3327e-01
input           0.000000 ± 0.000000        8.4607e-01 ± 2.3893e-01
nand           14.297103 ± 33.644669       1.2095e+00 ± 2.2113e-01
nor             2.889931 ± 9.089566

#### Prediction Margin

In [12]:
import torch
import numpy as np

print("\n--- Prediction Margin & Relative Error (GraphSAINT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Main loop: Prediction margin + relative error
# --------------------------------
class_margin_vals = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_margin_vals = []
overall_rel_errors  = []

epsilon = 1e-5  # small perturbation for verification

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    # Original logits
    with torch.no_grad():
        logits = model(graph, graph.ndata['features'])[test_idx]

    pred_class = int(torch.argmax(logits))
    pred_logit = logits[pred_class].item()

    # Second max logit
    other_logits = logits.clone()
    other_logits[pred_class] = -float('inf')
    second_max = other_logits.max().item()

    margin = pred_logit - second_max

    # Relative error verification
    perturbed_feats = graph.ndata['features'].clone()
    perturb_vec = torch.randn_like(perturbed_feats[test_idx]) * epsilon
    perturbed_feats[test_idx] += perturb_vec

    with torch.no_grad():
        logits_pert = model(graph, perturbed_feats)[test_idx]
    pred_logit_pert = logits_pert[pred_class].item()
    other_logits_pert = logits_pert.clone()
    other_logits_pert[pred_class] = -float('inf')
    second_max_pert = other_logits_pert.max().item()
    margin_pert = pred_logit_pert - second_max_pert

    rel_err = abs(margin - margin_pert) / (abs(margin_pert) + 1e-12)

    # Store
    class_margin_vals[class_name].append(margin)
    class_rel_errors[class_name].append(rel_err)
    overall_margin_vals.append(margin)
    overall_rel_errors.append(rel_err)

# --------------------------------
# 3) Reporting
# --------------------------------
print("\nClass-wise Prediction Margin (with Relative Error):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Margin ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    m_vals = class_margin_vals[cn]
    e_vals = class_rel_errors[cn]
    if m_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(m_vals), np.std(m_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_margin_vals:
    print("\nOverall Aggregated Prediction Margin:")
    print("Avg Margin: {:.4f} ± {:.4f}".format(np.mean(overall_margin_vals), np.std(overall_margin_vals)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated Prediction Margin:")
    print("No samples selected; overall metrics unavailable.")



--- Prediction Margin & Relative Error (GraphSAINT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Prediction Margin (with Relative Error):
Class                        Avg. Margin ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               9.1655 ± 4.9898          3.8058e-05 ± 9.7565e-05
input            16.2503 ± 2.4308          4.6727e-06 ± 4.0869e-06
nand              2.5752 ± 2.7748          1.4034e-04 ± 3.0827e-04
nor               4.9486 ± 2.4715          4.2161e-05 ± 6.

#### Adversarial Robustness Radius

In [14]:
import torch
import numpy as np

print("\n--- Adversarial Robustness Radius & Relative Error (GraphSAINT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Prediction wrapper for a node (replace its feature vector with x)
# --------------------------------
def f_for_sample(x, test_idx):
    """
    Given an input x (modified feature vector for the test node),
    returns the output (logit vector) for that node.
    """
    new_features = graph.ndata['features'].clone().detach()
    new_features[test_idx] = x
    with torch.no_grad():
        out = model(graph, new_features)
    return out[test_idx]

# --------------------------------
# 3) Adversarial radius along random directions + binary search
# --------------------------------
def adversarial_radius_for_sample(
    test_idx,
    initial_epsilon=1e-3,
    growth_factor=1.2,
    max_epsilon=10.0,
    bs_iters=10,
    num_trials=10,
):
    """
    Minimal perturbation norm required to change the prediction, estimated by:
      - Sampling random directions d (||d||=1)
      - Expanding epsilon multiplicatively until the prediction flips (or cap)
      - Binary-searching in the bracket to refine the boundary
    Returns the minimum over 'num_trials' sampled directions.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach()

    with torch.no_grad():
        base_out = model(graph, graph.ndata['features'])
        y0 = int(torch.argmax(base_out[test_idx]).item())

    def is_same(x):
        out = f_for_sample(x, test_idx)
        return int(torch.argmax(out).item()) == y0

    radii = []
    for _ in range(num_trials):
        d = torch.randn_like(x0)
        d = d / (torch.norm(d) + 1e-12)

        epsilon = initial_epsilon
        # Expand until flip or cap
        while epsilon < max_epsilon and is_same(x0 + epsilon * d):
            epsilon *= growth_factor

        if epsilon >= max_epsilon:
            candidate = max_epsilon
        else:
            # Binary search in [epsilon/growth_factor, epsilon]
            low = epsilon / growth_factor
            high = epsilon
            for _ in range(bs_iters):
                mid = 0.5 * (low + high)
                if is_same(x0 + mid * d):
                    low = mid
                else:
                    high = mid
            candidate = high
        radii.append(candidate)

    return float(min(radii))

# --------------------------------
# 4) Relative error verification (re-estimate with different config)
# --------------------------------
def adversarial_radius_relerr(test_idx):
    r1 = adversarial_radius_for_sample(
        test_idx,
        initial_epsilon=1e-3,
        growth_factor=1.2,
        max_epsilon=10.0,
        bs_iters=10,
        num_trials=10,
    )
    r2 = adversarial_radius_for_sample(
        test_idx,
        initial_epsilon=1e-3,
        growth_factor=1.3,  # different growth factor / search iters
        max_epsilon=10.0,
        bs_iters=12,
        num_trials=10,
    )
    rel_err = abs(r1 - r2) / (abs(r2) + 1e-12)
    return r1, rel_err

# --------------------------------
# 5) Main loop: compute ARR + relative error per class
# --------------------------------
class_adv_radius = {cn: [] for cn in class_names}
class_rel_errors  = {cn: [] for cn in class_names}
overall_adv_radius_vals = []
overall_rel_errors      = []

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    radius, rel_err = adversarial_radius_relerr(test_idx)

    class_adv_radius[class_name].append(radius)
    class_rel_errors[class_name].append(rel_err)
    overall_adv_radius_vals.append(radius)
    overall_rel_errors.append(rel_err)

# --------------------------------
# 6) Reporting
# --------------------------------
print("\nClass-wise Adversarial Robustness Radius (with Relative Error):")
header = "{:<12s} {:>27s} {:>30s}".format("Class", "Avg. Radius ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    r_vals = class_adv_radius[cn]
    e_vals = class_rel_errors[cn]
    if r_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(r_vals), np.std(r_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_adv_radius_vals:
    print("\nOverall Aggregated Adversarial Robustness Radius:")
    print("Avg Radius: {:.4f} ± {:.4f}".format(np.mean(overall_adv_radius_vals), np.std(overall_adv_radius_vals)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated Adversarial Robustness Radius:")
    print("No samples selected; overall metrics unavailable.")



--- Adversarial Robustness Radius & Relative Error (GraphSAINT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Adversarial Robustness Radius (with Relative Error):
Class                  Avg. Radius ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------
and               1.1797 ± 0.7354          4.9843e-01 ± 1.0521e+00
input             3.6205 ± 1.0731          2.3660e-01 ± 1.8090e-01
nand              0.5220 ± 0.8734          4.5337e-01 ± 5.1204e-01
nor               0.6590 ± 0.3118          2.9

#### Stability Under Input Noise

In [13]:
import torch
import numpy as np

print("\n--- Stability Under Input Noise & Relative Error (GraphSAINT, 100 Samples Per Class) ---\n")

# --------------------------------
# 0) Setup
# --------------------------------
model.eval()
class_names = list(label_encoder.classes_)
num_classes = len(class_names)
labels_np = graph.ndata['labels'].cpu().numpy()

# --------------------------------
# 1) Build selected_test_nodes: up to 100 per class from test_nid
# --------------------------------
rng = np.random.default_rng(42)
test_indices = test_nid.cpu().numpy()

selected_test_nodes = []
print("Sampling report:")
for c in range(num_classes):
    idxs = [int(i) for i in test_indices if labels_np[int(i)] == c]
    n_avail = len(idxs)
    if n_avail == 0:
        print(f"- {class_names[c]}: 0 available in test set  skipped.")
        continue
    if n_avail >= 100:
        chosen = rng.choice(idxs, size=100, replace=False)
        print(f"- {class_names[c]}: picked 100 from {n_avail} available.")
    else:
        chosen = np.array(idxs, dtype=np.int64)
        print(f"- {class_names[c]}: only {n_avail} available, taking all.")
    selected_test_nodes.extend(chosen.tolist())

selected_test_nodes = np.array(selected_test_nodes, dtype=np.int64)

# Final per-class counts
final_counts = {class_names[c]: int(np.sum(labels_np[selected_test_nodes] == c)) for c in range(num_classes)}
print("\nFinal counts in selected set:")
for name, cnt in final_counts.items():
    print(f"- {name}: {cnt}")

# --------------------------------
# 2) Helper: stability under Gaussian noise
# --------------------------------
def stability_for_sample(test_idx, sigma, num_samples):
    """
    Average L2 change in logits between clean and noisy versions of the node.
    """
    x0 = graph.ndata['features'][test_idx].clone().detach()
    with torch.no_grad():
        f_orig = model(graph, graph.ndata['features'])[test_idx]

    diffs = []
    for _ in range(num_samples):
        noise = sigma * torch.randn_like(x0)
        x_noisy = x0 + noise
        new_feats = graph.ndata['features'].clone().detach()
        new_feats[test_idx] = x_noisy
        with torch.no_grad():
            f_noisy = model(graph, new_feats)[test_idx]
        diffs.append(torch.norm(f_noisy - f_orig).item())
    return float(np.mean(diffs))

# --------------------------------
# 3) Main loop: stability + relative error
# --------------------------------
class_stability = {cn: [] for cn in class_names}
class_rel_errors = {cn: [] for cn in class_names}
overall_stability_vals = []
overall_rel_errors = []

sigma = 0.01
num_noise_samples = 20
relerr_resamples = 5

for test_idx in selected_test_nodes:
    label_idx = int(graph.ndata['labels'][test_idx].item())
    class_name = class_names[label_idx]

    stab_val = stability_for_sample(test_idx, sigma, num_noise_samples)

    # Relative error check: re-estimate stability with fresh noise
    re_vals = [stability_for_sample(test_idx, sigma, num_noise_samples)
               for _ in range(relerr_resamples)]
    avg_reval = float(np.mean(re_vals))
    rel_err = abs(stab_val - avg_reval) / (abs(avg_reval) + 1e-12)

    class_stability[class_name].append(stab_val)
    class_rel_errors[class_name].append(rel_err)
    overall_stability_vals.append(stab_val)
    overall_rel_errors.append(rel_err)

# --------------------------------
# 4) Reporting
# --------------------------------
print("\nClass-wise Stability Under Input Noise (with Relative Error):")
header = "{:<12s} {:>33s} {:>30s}".format("Class", "Avg. Stability ± Std", "Avg Rel. Error ± Std")
print(header)
print("-" * len(header))

for cn in class_names:
    s_vals = class_stability[cn]
    e_vals = class_rel_errors[cn]
    if s_vals:
        print("{:<12s} {:>11.4f} ± {:<10.4f} {:>15.4e} ± {:<10.4e}".format(
            cn, np.mean(s_vals), np.std(s_vals),
            np.mean(e_vals), np.std(e_vals)
        ))
    else:
        print("{:<12s} {:>11s} {:<10s} {:>15s} {:<10s}".format(cn, "-", "-", "-", "-"))

if overall_stability_vals:
    print("\nOverall Aggregated Stability Under Input Noise:")
    print("Avg Stability: {:.4f} ± {:.4f}".format(np.mean(overall_stability_vals), np.std(overall_stability_vals)))
    print("Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(overall_rel_errors), np.std(overall_rel_errors)))
else:
    print("\nOverall Aggregated Stability Under Input Noise:")
    print("No samples selected; overall metrics unavailable.")



--- Stability Under Input Noise & Relative Error (GraphSAINT, 100 Samples Per Class) ---

Sampling report:
- and: picked 100 from 5637 available.
- input: picked 100 from 475 available.
- nand: picked 100 from 527 available.
- nor: picked 100 from 567 available.
- not: picked 100 from 4478 available.
- or: picked 100 from 197 available.
- output: picked 100 from 266 available.
- xor: only 30 available, taking all.

Final counts in selected set:
- and: 100
- input: 100
- nand: 100
- nor: 100
- not: 100
- or: 100
- output: 100
- xor: 30

Class-wise Stability Under Input Noise (with Relative Error):
Class                     Avg. Stability ± Std           Avg Rel. Error ± Std
-----------------------------------------------------------------------------
and               0.4084 ± 0.1270          1.0617e-01 ± 7.4555e-02
input             0.2531 ± 0.0427          8.4543e-02 ± 6.3901e-02
nand              0.2225 ± 0.0744          8.5895e-02 ± 6.7289e-02
nor               0.2112 ± 0.0391     